In [36]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL

In [37]:
from config import Config

In [47]:
conf = Config()

conf.dataset = 'nyc'
conf.post_value_updates()

In [48]:
conf.dataset_prefix

'porto_20200'

In [41]:
config.dataset_cell_file

'/home/sagemaker-user/TrajCL/data/porto_20200_cell100_cellspace.pkl'

In [7]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# Load data.
df = ddl.load_as_spark(schema="datascience_scratchpad", table="nyc_train_filtered")

:: loading settings :: url = jar:file:/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sagemaker-user/.ivy2/cache
The jars for the packages stored in: /home/sagemaker-user/.ivy2/jars
io.delta#delta-sharing-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-467ad064-395c-4398-9ad9-011748b3d9cc;1.0
	confs: [default]
	found io.delta#delta-sharing-spark_2.12;3.2.0 in central
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-sharing-client_2.12;1.0.5 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-o

In [17]:
from pyspark.sql.functions import col
candidate_df = df.filter(col("traj_id").contains("4474029_")).toPandas()

In [19]:
candidate_df.head()

,traj_id,timestamps_filtered,wgs_seq_filtered,merc_seq_filtered,trajlen
0,4474029_2025-02-14,"[2025-02-14 04:45:18, 2025-02-14 05:40:37, 202...","[[-74.2201211, 40.842971], [-74.2201674, 40.84...","[[-8262146.0874671005, 4989207.440164468], [-8...",30
1,4474029_2019-07-27,"[2019-07-27 00:00:19, 2019-07-27 00:12:05, 201...","[[-74.2198396, 40.8427089], [-74.2201477, 40.8...","[[-8262114.751030441, 4989168.872291069], [-82...",88
2,4474029_2025-06-01,"[2025-06-01 03:37:20, 2025-06-01 07:10:11, 202...","[[-74.219858, 40.8430117], [-74.2200431, 40.84...","[[-8262116.799309071, 4989213.429161161], [-82...",25
3,4474029_2020-09-13,"[2020-09-13 00:11:59, 2020-09-13 03:28:21, 202...","[[-74.2200829, 40.8430845], [-74.2198778, 40.8...","[[-8262141.835062551, 4989224.141675475], [-82...",32
4,4474029_2020-03-12,"[2020-03-12 06:57:58, 2020-03-12 07:24:17, 202...","[[-74.2199927, 40.8429431], [-74.2200895, 40.8...","[[-8262131.7940444825, 4989203.334687288], [-8...",29


In [35]:
config.dataset_cell_file

'/home/sagemaker-user/TrajCL/data/porto_20200_cell100_cellspace.pkl'

In [65]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/snapshots/nyc_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_6260/2792526184.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTB(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): SpatialMSM(


In [66]:
from utils.traj import *
import pickle

from torch.nn.utils.rnn import pad_sequence
embs = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell500_embdim256_embs.pkl", 'rb')).to('cpu').detach() # tensor
cellspace = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell500_cellspace.pkl", 'rb'))

def infer(traj):
    traj_cell, traj_p = zip(*[merc2cell2(t, cellspace) for t in traj])
    traj_emb_p = [torch.tensor(generate_spatial_features(t, cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell = [embs[list(t)] for t in traj_cell]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_cell)), dtype = torch.long, device = device)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len)
    return traj_embs



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [67]:
output = infer(candidates_traj)

In [68]:
output.shape

torch.Size([473, 256])

In [71]:
import numpy as np
from sklearn.preprocessing import normalize
# Normalize the rows (L2 normalization)
output_normalized = normalize(output.cpu().detach().numpy(), axis=1)

# Compute cosine similarity: dot product of normalized vectors
similarity_matrix = np.dot(output_normalized, output_normalized.T)

In [73]:
import numpy as np

# Assume similarity_matrix is your m x m cosine similarity matrix
# Optional: set diagonal to -inf to ignore self-similarity
np.fill_diagonal(similarity_matrix, -np.inf)

# Get top 5 similar indices for each row
top5_indices = np.argsort(-similarity_matrix, axis=1)[:, :5]

print(top5_indices)

[[ 33 466 316 412 165]
 [ 99 245  71 429 333]
 [158 472 294  91  44]
 ...
 [461 435 379 142 271]
 [400 236 234 297 177]
 [128 322 172 357 207]]


In [80]:
import numpy as np

# Assume similarity_matrix is your m x m cosine similarity matrix
# Optional: set diagonal to -inf to ignore self-similarity
np.fill_diagonal(similarity_matrix, -np.inf)

# Get top 5 similar indices for each row
bottom5_indices = np.argsort(similarity_matrix, axis=1)[:, :5]

print(bottom5_indices)

[[  0  37 441 372 180]
 [  1 211 162 247 101]
 [  2 101 173 295  89]
 ...
 [470 101 209  22  89]
 [471 211 101 162 298]
 [472 295  89 173   4]]


In [79]:
# similar trajactories
traj_id_list = list(candidate_df['traj_id'])
for i in range(10):
    print("{} : ".format(traj_id_list[i]), [(traj_id_list[x], similarity_matrix[i][x]) for x in top5_indices[i]])



4474029_2025-02-14 :  [('4474029_2025-03-21', 0.951891), ('4474029_2024-12-24', 0.9515693), ('4474029_2024-11-23', 0.9484984), ('4474029_2025-04-09', 0.9475976), ('4474029_2024-12-03', 0.94156325)]
4474029_2019-07-27 :  [('4474029_2019-08-04', 0.99469346), ('4474029_2019-11-17', 0.9936477), ('4474029_2019-06-02', 0.99351305), ('4474029_2019-11-19', 0.9921478), ('4474029_2019-06-16', 0.9915563)]
4474029_2025-06-01 :  [('4474029_2022-04-23', 0.9245522), ('4474029_2025-02-04', 0.9022228), ('4474029_2022-08-30', 0.90073687), ('4474029_2025-03-08', 0.8958865), ('4474029_2022-09-26', 0.89439297)]
4474029_2020-09-13 :  [('4474029_2019-08-31', 0.96266145), ('4474029_2019-11-02', 0.9570831), ('4474029_2025-03-20', 0.9440542), ('4474029_2019-07-21', 0.9371742), ('4474029_2019-08-19', 0.93653125)]
4474029_2020-03-12 :  [('4474029_2020-01-03', 0.9304388), ('4474029_2020-03-04', 0.9237604), ('4474029_2020-02-20', 0.914265), ('4474029_2020-03-03', 0.90973747), ('4474029_2020-02-19', 0.8975645)]
4474

In [81]:
# similar trajactories
traj_id_list = list(candidate_df['traj_id'])
for i in range(10):
    print("{} : ".format(traj_id_list[i]), [(traj_id_list[x], similarity_matrix[i][x]) for x in bottom5_indices[i]])

4474029_2025-02-14 :  [('4474029_2025-02-14', -inf), ('4474029_2020-11-01', -0.01639717), ('4474029_2020-02-11', 0.015235383), ('4474029_2019-12-20', 0.01625728), ('4474029_2020-02-07', 0.021524005)]
4474029_2019-07-27 :  [('4474029_2019-07-27', -inf), ('4474029_2020-11-28', -0.033838633), ('4474029_2023-10-21', -0.020210706), ('4474029_2023-08-27', 0.01195579), ('4474029_2020-01-09', 0.01954792)]
4474029_2025-06-01 :  [('4474029_2025-06-01', -inf), ('4474029_2020-01-09', -0.00023288565), ('4474029_2020-03-03', 0.044660028), ('4474029_2020-02-05', 0.060766675), ('4474029_2019-12-16', 0.06908514)]
4474029_2020-09-13 :  [('4474029_2020-09-13', -inf), ('4474029_2020-11-28', -0.05460479), ('4474029_2020-01-09', -0.04002695), ('4474029_2020-02-07', -0.024458436), ('4474029_2020-01-17', -0.0017153919)]
4474029_2020-03-12 :  [('4474029_2020-03-12', -inf), ('4474029_2020-01-20', -0.1571126), ('4474029_2025-07-05', -0.13545239), ('4474029_2022-06-12', -0.12833218), ('4474029_2023-10-14', -0.121